In [1]:
import pandas as pd
import numpy as np
import os as os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.util import ngrams
import nltk
from arabica import arabica_freq
from unidecode import unidecode
import re
import plotly.express as px
import texthero as hero
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.naive_bayes import BernoulliNB
import spacy
import re
import nlp
import warnings
import statsmodels.api as sm
from statsmodels.formula.api import ols


warnings.simplefilter(action='ignore', category=FutureWarning)

2023-09-25 19:03:46.059975: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 19:03:46.490029: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 19:03:46.492517: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 19:03:49.616172: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('sentimento - base3.csv')

In [3]:
df.head()

,id_issue,mes_chat,SENTIMENTO,TIPO_MSG,hierarquia_4,ds_cluster_sociodemographic,ds_message,flag_negativo_indecx
0,15388962,2023-04-01,neutro,duvida,Vencimento fatura,"0 - Mulher, Casada, 25 anos ou mais, cidade Pe...","['Que dia minha fatura fecha?','Ainda tenho dú...",0
1,17497395,2023-07-01,neutro,duvida,Dúvidas sobre pagamento de fatura,"11 - Homem, Solteiro, Até 24 anos, cidade Pequ...","['Quero pagar os 100 reais que eu tirei','Mais...",0
2,15213047,2023-04-01,neutro,solicitacao,Não reconhecimento de compra no crédito,"17 - Mulher, 25 a 40 anos, Sudeste e Sul","['Apareceu uma compra que eu não fiz','Dúvida ...",0
3,17117301,2023-06-01,neutro,problema,Parcelamento de fatura,"8 - Mulher, Solteira, Até 24 anos, cidade Pequ...",['No meu limite está escrito que está utilizad...,0
4,17345996,2023-07-01,negativo,problema,Dúvidas PIX,"10 - Mulher, Solteira, Até 24 anos, cidade Peq...","['Pix','E merda o will nunca teve essas coisas...",0


In [4]:
# def classif_emoji(text):
#     '''
#     Replace emojis with word emoji
#     '''
#     regrex_pattern = re.compile(pattern="["
#                                 u"\U0001F600-\U0001F64F"  # emoticons
#                                 u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#                                 u"\U0001F680-\U0001F6FF"  # transport & map symbols
#                                 u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                                 "]+", flags=re.UNICODE)
#     return regrex_pattern.sub(r'emoji ', str(text))

def clean_text(x):
    #x = unidecode(x)
    x = x.strip()
    x = x.lower()
    x = re.sub(r'\t', ' ', x)
    x = re.sub(r'http\S+', '', x)  # remove url
    x = re.sub(r'screenshot sent', 'screenshotsent', x)
    x = re.sub(r'will bank', 'willbank', x)
    x = re.sub(r'(nbsp\;|[rl]dquo\;|quot\;|acute\;|tilde\;|circ\;|grave\;|cedil\;|\&|\/)', '', x)  # replace special characters
    x = x.replace('&ccedil;','ç')
    x = x.replace('&atilde;;','ã')
    x = x.replace('&otilde;;','õ')
    #x = re.sub(r'(\d{1,}|\!|\?|\,|\.|\\n|\:|\;|\`|\´|\-|\_)', ' ', x)  # replace punctuation and remove numbers
    x = x.replace('  ',' ')
    x = x.replace('\n','')
    x = x.strip()
    return x


In [5]:
len(df)

14906

In [6]:
# não vou remover emojis
df['ds_message_tratada'] = df['ds_message'].apply(clean_text)

In [7]:
df['ds_message_tratada'][4]

"['pix','e merda o will nunca teve essas coisas','estou precisando fazer o pix e não estou conseguindo','desde ontem','não dá pra fazer pix','mais opções']"

In [8]:
df['n_palavras'] = df['ds_message_tratada'].str.count(' ')

In [9]:
df['n_palavras_t'] = np.where(df.n_palavras >= 20, 20, df.n_palavras)

In [10]:
pd.crosstab(df.n_palavras_t, df.SENTIMENTO)

SENTIMENTO,negativo,neutro,positivo
n_palavras_t,,,
2,0,0,1
3,0,2,0
4,1,3,1
5,2,10,1
6,2,8,0
7,0,8,1
8,2,8,1
9,2,6,0
10,2,10,1


In [13]:
from LeIA import SentimentIntensityAnalyzer 


# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()
# Create a new column in the dataframe to store the sentiment scores
df["sentiment"] = 0.0

# Iterate over the rows of the dataframe
for index, row in df.iterrows():
  # Use the analyzer to predict the sentiment of the text
  scores = analyzer.polarity_scores(row["ds_message"])
  # Store the compound score in the new column
  df.at[index, "sentiment"] = scores["compound"]

In [60]:
exemplo_base = df[df['SENTIMENTO'].notnull()]
len(exemplo_base)

645

In [12]:
stopwords_will= ['da', 'em', 'você', 'de', 'ao', 'os', 'que', 'e', 'eu', 'o', 'estou', 'ele','ela','pra', 'pro', 'entao', 'para', 'voce',
                 'pra', 'tá', 'né', 'ah', 'aí', 'dá', 'ó', 'mim', 'então', 'aqui', 'tô', 'pro',
            'seu', 'sua', 'por', 'sobre', 'mais', 'uma', 'um','como', 'meu', 'com', 'outro','outra','das','dos','foi', 'fiz','pelo','tem','mas',
            'este', 'esse','ta', 'to', 'ai', 'la', 'lo', 'lhe', 'ne','pra', 'tá', 'né', 'ah', 'aí', 'dá', 'ó', 'mim', 'então', 'aqui', 'tô', 'pro', 'isso']

In [14]:
exemplo_base['SENTIMENTO_label'] = np.where(exemplo_base['SENTIMENTO'] == "negativo", 0, 
                                            np.where(exemplo_base['SENTIMENTO'] == "neutro", 1, 2 ))

/tmp/ipykernel_972980/2720231541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [31]:
exemplo_base['SENTIMENTO_label'].value_counts()

0    265
1    258
2    122
Name: SENTIMENTO_label, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split

vetorizador = TfidfVectorizer()
X = vetorizador.fit_transform(exemplo_base['ds_message'])
y = exemplo_base.SENTIMENTO_label


x_train, x_test, y_train, y_test = train_test_split(X
                                                    , exemplo_base.SENTIMENTO_label
                                                    , random_state=42, test_size=0.20)

In [16]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn import metrics

#INCLUIR PARAMETROS DO MELHOR MODELO
clf = xgb.XGBClassifier(objective="multi:softprob", max_depth = 3, num_round = 437, eta = 0.02, subsample = 0.8,  alpha = 0.0, colsample_bytree = 0.40, gamma = 3.7, random_state = 42)
#lambda = 1.644,
clf = clf.fit(x_train,y_train)

y_predicted = clf.predict(x_test)
pd.crosstab(y_predicted, y_test)

[22:12:47] WARNING: ../src/learner.cc:767: 
Parameters: { "num_round" } are not used.



SENTIMENTO_label,0,1,2
row_0,,,
0,40,9,3
1,15,38,10
2,3,0,11


In [17]:
scores = cross_val_score(clf, x_train, y_train, cv=5, scoring='f1_macro')
print("%0.2f F1 with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

[22:12:58] WARNING: ../src/learner.cc:767: 
Parameters: { "num_round" } are not used.

[22:12:59] WARNING: ../src/learner.cc:767: 
Parameters: { "num_round" } are not used.

[22:13:01] WARNING: ../src/learner.cc:767: 
Parameters: { "num_round" } are not used.

[22:13:03] WARNING: ../src/learner.cc:767: 
Parameters: { "num_round" } are not used.

[22:13:05] WARNING: ../src/learner.cc:767: 
Parameters: { "num_round" } are not used.

0.69 F1 with a standard deviation of 0.07


In [26]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predicted))


              precision    recall  f1-score   support

           0       0.77      0.69      0.73        58
           1       0.60      0.81      0.69        47
           2       0.79      0.46      0.58        24

    accuracy                           0.69       129
   macro avg       0.72      0.65      0.67       129
weighted avg       0.71      0.69      0.69       129



In [32]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn import metrics

#INCLUIR PARAMETROS DO MELHOR MODELO
clf = xgb.XGBClassifier(objective="multi:softprob", max_depth = 10)
#lambda = 1.644,
clf = clf.fit(x_train,y_train)

y_predicted = clf.predict(x_test)
pd.crosstab(y_predicted, y_test)

SENTIMENTO_label,0,1,2
row_0,,,
0,42,10,3
1,11,34,4
2,5,3,17


In [33]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.76      0.72      0.74        58
           1       0.69      0.72      0.71        47
           2       0.68      0.71      0.69        24

    accuracy                           0.72       129
   macro avg       0.71      0.72      0.72       129
weighted avg       0.72      0.72      0.72       129



In [34]:
#INCLUIR PARAMETROS DO MELHOR MODELO
clf = xgb.XGBClassifier(objective="multi:softprob", max_depth = 30)
#lambda = 1.644,
clf = clf.fit(x_train,y_train)

y_predicted = clf.predict(x_test)
pd.crosstab(y_predicted, y_test)

SENTIMENTO_label,0,1,2
row_0,,,
0,44,8,3
1,10,36,5
2,4,3,16


In [35]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.80      0.76      0.78        58
           1       0.71      0.77      0.73        47
           2       0.70      0.67      0.68        24

    accuracy                           0.74       129
   macro avg       0.73      0.73      0.73       129
weighted avg       0.75      0.74      0.74       129



In [36]:
#INCLUIR PARAMETROS DO MELHOR MODELO
clf = xgb.XGBClassifier(objective="multi:softprob", max_depth = 50)
#lambda = 1.644,
clf = clf.fit(x_train,y_train)

y_predicted = clf.predict(x_test)
pd.crosstab(y_predicted, y_test)

SENTIMENTO_label,0,1,2
row_0,,,
0,45,9,3
1,10,35,6
2,3,3,15


In [37]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.79      0.78      0.78        58
           1       0.69      0.74      0.71        47
           2       0.71      0.62      0.67        24

    accuracy                           0.74       129
   macro avg       0.73      0.72      0.72       129
weighted avg       0.74      0.74      0.74       129



In [38]:
from sklearn.neural_network import MLPClassifier

vetorizador = TfidfVectorizer()
X = vetorizador.fit_transform(exemplo_base['ds_message_tratada'])
y = exemplo_base.SENTIMENTO_label

x_train, x_test, y_train, y_test = train_test_split(X
                                                    , exemplo_base.SENTIMENTO_label
                                                    , random_state=42, test_size=0.20)

clf = MLPClassifier(solver='lbfgs', alpha=0.0001, hidden_layer_sizes = 100, random_state=1) #alpha=0.0001

clf = clf.fit(x_train, y_train)
y_predicted = clf.predict(x_test)
pd.crosstab(y_predicted, y_test)

SENTIMENTO_label,0,1,2
row_0,,,
0,45,4,5
1,12,42,10
2,1,1,9


In [39]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.83      0.78      0.80        58
           1       0.66      0.89      0.76        47
           2       0.82      0.38      0.51        24

    accuracy                           0.74       129
   macro avg       0.77      0.68      0.69       129
weighted avg       0.77      0.74      0.73       129



In [42]:
vetorizador = TfidfVectorizer()
X = vetorizador.fit_transform(exemplo_base['ds_message_tratada'])
y = exemplo_base.SENTIMENTO_label

x_train, x_test, y_train, y_test = train_test_split(X
                                                    , exemplo_base.SENTIMENTO_label
                                                    , random_state=42, test_size=0.20)

clf = MLPClassifier(solver='lbfgs', alpha=0.0001, hidden_layer_sizes = 200, random_state=1) #alpha=0.0001

clf = clf.fit(x_train, y_train)
y_predicted = clf.predict(x_test)
pd.crosstab(y_predicted, y_test)

SENTIMENTO_label,0,1,2
row_0,,,
0,44,4,10
1,13,41,4
2,1,2,10


In [43]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.76      0.76      0.76        58
           1       0.71      0.87      0.78        47
           2       0.77      0.42      0.54        24

    accuracy                           0.74       129
   macro avg       0.74      0.68      0.69       129
weighted avg       0.74      0.74      0.73       129



In [44]:
vetorizador = TfidfVectorizer()
X = vetorizador.fit_transform(exemplo_base['ds_message_tratada'])
y = exemplo_base.SENTIMENTO_label

x_train, x_test, y_train, y_test = train_test_split(X
                                                    , exemplo_base.SENTIMENTO_label
                                                    , random_state=42, test_size=0.20)

clf = MLPClassifier(solver='lbfgs', alpha=0.0001, hidden_layer_sizes = 50, random_state=1) #alpha=0.0001

clf = clf.fit(x_train, y_train)
y_predicted = clf.predict(x_test)
pd.crosstab(y_predicted, y_test)

SENTIMENTO_label,0,1,2
row_0,,,
0,42,4,7
1,14,40,5
2,2,3,12


In [45]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.79      0.72      0.76        58
           1       0.68      0.85      0.75        47
           2       0.71      0.50      0.59        24

    accuracy                           0.73       129
   macro avg       0.73      0.69      0.70       129
weighted avg       0.73      0.73      0.72       129



- Combinar MLP com 100 hl + XGBoost com max_depth = 30 + score Leya

## Início Stacking

In [46]:
mlp = MLPClassifier(solver='lbfgs', alpha=0.0001, hidden_layer_sizes = 100, random_state=1) #alpha=0.0001
mlp = mlp.fit(x_train, y_train)
tree = xgb.XGBClassifier(objective="multi:softprob", max_depth = 30)
tree = tree.fit(x_train, y_train)


In [54]:
x_test2_pr_mlp_0 = mlp.predict_proba(x_test)[:, 0]
x_test2_pr_mlp_1 = mlp.predict_proba(x_test)[:, 1]
x_test2_pr_mlp_2 = mlp.predict_proba(x_test)[:, 2]

x_test2_pr_tree_0 = tree.predict_proba(x_test)[:, 0]
x_test2_pr_tree_1 = tree.predict_proba(x_test)[:, 1]
x_test2_pr_tree_2 = tree.predict_proba(x_test)[:, 2]


In [56]:
x_test2 = pd.DataFrame()

x_test2['prob_mlp_0'] = x_test2_pr_mlp_0
x_test2['prob_mlp_1'] = x_test2_pr_mlp_1
x_test2['prob_mlp_2'] = x_test2_pr_mlp_2

x_test2['prob_tree_0'] = x_test2_pr_tree_0
x_test2['prob_tree_1'] = x_test2_pr_tree_1
x_test2['prob_tree_2'] = x_test2_pr_tree_2

In [61]:
x_test2 = x_test2.join(exemplo_base[['n_palavras_t', 'sentiment']])

In [65]:
x_test2['n_palavras_t'] = x_test2.n_palavras_t.fillna(0.0)
x_test2['sentiment'] = x_test2.sentiment.fillna(0.0)

In [67]:
x_test2.tail(5)

,prob_mlp_0,prob_mlp_1,prob_mlp_2,prob_tree_0,prob_tree_1,prob_tree_2,n_palavras_t,sentiment
124,1.055059e-06,9.999989e-01,2.698293e-15,0.686580,0.309524,0.003896,0.0,0.0
125,9.999998e-01,1.881597e-07,6.263854e-27,0.856470,0.139792,0.003738,0.0,0.0
126,1.414555e-03,9.985854e-01,2.352575e-16,0.011962,0.984616,0.003422,20.0,0.0
127,5.596574e-26,8.983122e-23,1.000000e+00,0.001050,0.003727,0.995224,0.0,0.0
128,1.950403e-02,9.804960e-01,2.651671e-14,0.838743,0.156760,0.004497,0.0,0.0


In [68]:
x_train2_pr_mlp_0 = mlp.predict_proba(x_train)[:, 0]
x_train2_pr_mlp_1 = mlp.predict_proba(x_train)[:, 1]
x_train2_pr_mlp_2 = mlp.predict_proba(x_train)[:, 2]

x_train2_pr_tree_0 = tree.predict_proba(x_train)[:, 0]
x_train2_pr_tree_1 = tree.predict_proba(x_train)[:, 1]
x_train2_pr_tree_2 = tree.predict_proba(x_train)[:, 2]

x_train2 = pd.DataFrame()

x_train2['prob_mlp_0'] = x_train2_pr_mlp_0
x_train2['prob_mlp_1'] = x_train2_pr_mlp_1
x_train2['prob_mlp_2'] = x_train2_pr_mlp_2

x_train2['prob_tree_0'] = x_train2_pr_tree_0
x_train2['prob_tree_1'] = x_train2_pr_tree_1
x_train2['prob_tree_2'] = x_train2_pr_tree_2

x_train2 = x_train2.join(exemplo_base[['n_palavras_t', 'sentiment']])

x_train2['n_palavras_t'] = x_train2.n_palavras_t.fillna(0.0)
x_train2['sentiment'] = x_train2.sentiment.fillna(0.0)

x_train2.head(5)

,prob_mlp_0,prob_mlp_1,prob_mlp_2,prob_tree_0,prob_tree_1,prob_tree_2,n_palavras_t,sentiment
0,1.000000e+00,3.225956e-17,1.970928e-40,0.996199,0.000344,0.003457,6.0,0.0000
1,4.292899e-04,9.995707e-01,5.947390e-14,0.012131,0.987070,0.000799,20.0,-0.4215
2,5.628713e-11,1.000000e+00,5.789175e-22,0.003332,0.994056,0.002612,11.0,-0.2960
3,1.000000e+00,1.005106e-11,1.440902e-42,0.913423,0.083614,0.002963,20.0,0.3400
4,2.218729e-12,1.000000e+00,4.727892e-10,0.000515,0.990211,0.009274,20.0,-0.7003


In [72]:
clf = xgb.XGBClassifier(objective="multi:softprob", max_depth = 10, random_state = 42)
clf = clf.fit(x_train2, y_train)

In [75]:
y_predicted = []
y_predicted = clf.predict(x_test2)
pd.crosstab(y_predicted, y_test)

SENTIMENTO_label,0,1,2
row_0,,,
0,45,4,5
1,12,42,10
2,1,1,9


In [76]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.83      0.78      0.80        58
           1       0.66      0.89      0.76        47
           2       0.82      0.38      0.51        24

    accuracy                           0.74       129
   macro avg       0.77      0.68      0.69       129
weighted avg       0.77      0.74      0.73       129

